In [52]:
import geopandas as gpd
from pathlib import Path
import rasterio

parcels_df = None
raw_data = None

def load_parcels_data():
    global parcels_df, raw_data
    raw_data = Path("../data/raw_data/")
    parcels_df = gpd.read_file(raw_data / "nl_parcels_subset_small.gpkg")

def add_centroid():
    parcels_df["centroid"] = parcels_df.apply(lambda row : row['geometry'].centroid , axis=1)

def add_soc_and_centroid():
    carbon_tif_path = raw_data / "soil_log_organic_carbon_content_2020.tif"
    with rasterio.open(carbon_tif_path) as f:
        raster_stats = f.statistics(bidx=1)

        add_centroid()
        centroid_list = parcels_df["centroid"]

        new_df = gpd.GeoDataFrame(geometry = centroid_list,crs=parcels_df.crs)
        new_df = new_df.to_crs(f.crs)

        coord_list = [(x, y) for x, y in zip(new_df["geometry"].x, new_df["geometry"].y)]
        parcels_df["soc"] = [raster_stats.max - x for x in f.sample(coord_list)]

def get_in_batch(st, end):
    if parcels_df is None:
        return None
    return parcels_df[st:end]

In [53]:
load_parcels_data()

In [54]:
df = get_in_batch(0,5)
df.head()

,objectid,area,crop_type,geometry
0,1526170,51374.322841,winter_common_soft_wheat,"MULTIPOLYGON (((3.79995 51.47216, 3.80034 51.4..."
1,1526353,96633.566485,chicory_chicories,"MULTIPOLYGON (((3.76622 51.49420, 3.76622 51.4..."
2,1526354,3829.956679,sugar_beet,"MULTIPOLYGON (((3.78401 51.49023, 3.78425 51.4..."
3,1526436,8116.934914,nurseries_nursery,"MULTIPOLYGON (((3.98940 51.46791, 3.98940 51.4..."
4,1526437,94258.776946,grain_maize_corn_popcorn,"MULTIPOLYGON (((3.98666 51.47670, 3.98666 51.4..."


In [55]:
res = df.to_json()
jval = json.loads(res)
add_soc_and_centroid()
df = get_in_batch(0,5)
df.head()

,objectid,area,crop_type,geometry,centroid,soc
0,1526170,51374.322841,winter_common_soft_wheat,"MULTIPOLYGON (((3.79995 51.47216, 3.80034 51.4...",POINT (3.79847 51.47176),[11.0]
1,1526353,96633.566485,chicory_chicories,"MULTIPOLYGON (((3.76622 51.49420, 3.76622 51.4...",POINT (3.76257 51.49440),[13.0]
2,1526354,3829.956679,sugar_beet,"MULTIPOLYGON (((3.78401 51.49023, 3.78425 51.4...",POINT (3.78487 51.49023),[13.0]
3,1526436,8116.934914,nurseries_nursery,"MULTIPOLYGON (((3.98940 51.46791, 3.98940 51.4...",POINT (3.99000 51.46835),[14.0]
4,1526437,94258.776946,grain_maize_corn_popcorn,"MULTIPOLYGON (((3.98666 51.47670, 3.98666 51.4...",POINT (3.98882 51.47669),[10.0]


In [61]:
req = []
for i in df.index:
    item = dict(jval['features'][i]['properties'])
    item['soc'] = df.iloc[i]['soc'][0]
    item['centroid']  = { "type": "Point", "coordinates": [df.iloc[i]['centroid'].x,df.iloc[i]['centroid'].y] }
    req.append(item)

In [62]:
for r in req:
    print(r)

{'objectid': 1526170, 'area': 51374.3228411179, 'crop_type': 'winter_common_soft_wheat', 'soc': 11.0, 'centroid': {'type': 'Point', 'coordinates': [3.7984652985271317, 51.471757061187844]}}
{'objectid': 1526353, 'area': 96633.5664848345, 'crop_type': 'chicory_chicories', 'soc': 13.0, 'centroid': {'type': 'Point', 'coordinates': [3.7625663803791913, 51.49439887043063]}}
{'objectid': 1526354, 'area': 3829.95667879842, 'crop_type': 'sugar_beet', 'soc': 13.0, 'centroid': {'type': 'Point', 'coordinates': [3.7848685336851986, 51.490228820648674]}}
{'objectid': 1526436, 'area': 8116.93491382445, 'crop_type': 'nurseries_nursery', 'soc': 14.0, 'centroid': {'type': 'Point', 'coordinates': [3.990002187107968, 51.46834839829888]}}
{'objectid': 1526437, 'area': 94258.7769456627, 'crop_type': 'grain_maize_corn_popcorn', 'soc': 10.0, 'centroid': {'type': 'Point', 'coordinates': [3.9888207531432034, 51.47668892662685]}}


In [ ]:
import requests
url = 'https://localhost:8080/parcels/add_parcels'
for r in req:
    x = requests.post(url, data = r, headers = {"Accept": "application/json"})
    print(x.text)